In [54]:
import numpy as np
import pandas as pd
import glob
import os

### Step one:
#### Adapted from [StackOverflow](https://stackoverflow.com/questions/41857659/python-pandas-add-filename-column-csv)
#### Merge 505 company files into one file, append filename as symbol

In [79]:
globbed_files = glob.glob("data/*.csv")

In [56]:
frame = pd.DataFrame()

In [57]:
for csv in globbed_files:
    frame_t = pd.read_csv(csv, parse_dates=["Date"])
    filename = os.path.basename(csv)
    frame_t['symbol'] = os.path.splitext(filename)[0]
    frame = pd.concat([frame, frame_t], ignore_index=True)

In [58]:
frame

,Date,Open,High,Low,Close,Adj Close,Volume,symbol
0,1990-02-16,0.000000,0.079861,0.073785,0.077257,0.057775,940636800.0,CSCO
1,1990-02-20,0.000000,0.079861,0.074653,0.079861,0.059723,151862400.0,CSCO
2,1990-02-21,0.000000,0.078993,0.075521,0.078125,0.058424,70531200.0,CSCO
3,1990-02-22,0.000000,0.081597,0.078993,0.078993,0.059073,45216000.0,CSCO
4,1990-02-23,0.000000,0.079861,0.078125,0.078559,0.058749,44697600.0,CSCO
...,...,...,...,...,...,...,...,...
3918152,2021-01-25,27.639999,28.650000,27.549999,28.110001,28.110001,6237400.0,PPL
3918153,2021-01-26,28.120001,28.430000,27.840000,28.250000,28.250000,3469800.0,PPL
3918154,2021-01-27,28.049999,28.440001,27.780001,27.799999,27.799999,6147000.0,PPL
3918155,2021-01-28,27.889999,28.610001,27.770000,28.129999,28.129999,3754600.0,PPL


In [273]:
start_date = '2018-01-29'
end_date = '2021-01-29'

In [274]:
df = frame.loc[(frame['Date'] >= start_date) & (frame['Date'] <= end_date)]

In [275]:
df = df.sort_values(by=['symbol']).reset_index(drop=True)

### Step two:
#### Do an inner join with marketcap.csv on symbol

In [276]:
df_marketcap = pd.read_csv('marketcap.csv')

In [277]:
merged_r = df.merge(df_marketcap, on=['symbol'], how='right')

In [278]:
merged_r[merged_r['Close'].isna()]['symbol'].unique()

array(['BRK.B', 'BF.B', 'HWM', 'MPWR'], dtype=object)

In [279]:
merged_l = df.merge(df_marketcap, on=['symbol'], how='left')
merged_l[merged_l['marketcap'].isna()]['symbol'].unique()

array(['BF-B', 'BRK-B', 'FTI'], dtype=object)

After looking up the news on the Internet, we realized that FTI is replaced by MPWR since Feb. 12th 2021. However, since our data in the the period of 2018-01-29 to 2021-01-29, we will use FTI instead of MPWR.
In addition, replace 'BF.B' and 'BRK.B' with 'BF-B' and 'BRK-B' in marketcap.csv

In [280]:
df_marketcap = df_marketcap[df_marketcap.symbol!='MPWR']

In [281]:
df_marketcap.loc[df_marketcap['symbol']=='BF.B','symbol'] = 'BF-B'

In [282]:
df_marketcap.loc[df_marketcap['symbol']=='BRK.B','symbol'] = 'BRK-B'

retrieve the market cap of FTI in google sheet by using function `=GOOGLEFINANCE("NYSE:FTI", "marketcap")`

In [283]:
FTI_marketcap_row = {'symbol':'FTI', 'name':'TechnipFMC', 'sector':'Energy', 'marketcap':3945801943}

In [284]:
df_marketcap = df_marketcap.append(FTI_marketcap_row, ignore_index=True)

In [285]:
df_marketcap.to_csv('marketcap_preprocessed.csv')

We also realize that data of 'HWM' is corrupted, thus we need to replace the data in df

In [286]:
from pandas_datareader import data

In [287]:
hwm = data.DataReader("HWM", 
                       start='2018-01-29', 
                       end='2021-01-29', 
                       data_source='yahoo')

In [288]:
hwm

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-01-29,30.910000,30.170000,30.830000,30.260000,2462700.0,29.733521
2018-01-30,30.180000,29.379999,30.020000,29.920000,4431300.0,29.399433
2018-01-31,30.510000,29.770000,30.000000,30.059999,3403200.0,29.536999
2018-02-01,30.389999,29.719999,29.750000,30.110001,2808400.0,29.645300
2018-02-02,29.760000,28.830000,29.760000,29.110001,7621800.0,28.660734
...,...,...,...,...,...,...
2021-01-25,26.610001,25.340000,26.480000,25.830000,7595900.0,25.830000
2021-01-26,26.280001,24.910000,26.139999,25.000000,7258600.0,25.000000
2021-01-27,24.450001,22.910000,24.389999,23.600000,8144000.0,23.600000


In [289]:
hwm = hwm.reset_index()

In [290]:
hwm['symbol']='HWM'

In [291]:
df = df[df.symbol!='HWM']

In [292]:
df = pd.concat([df,hwm], ignore_index=True)

In [293]:
df = df.sort_values(by=['symbol','Date']).reset_index(drop=True)

In [294]:
df = df.rename(columns={'Adj Close':'Adj_Close'})

In [295]:
df_short = df.drop(columns=['High','Low','Open','Close'])

In [296]:
df_short.to_csv()

,Date,Adj_Close,Volume,symbol
0,2018-01-29,72.631302,1321900.0,A
1,2018-01-30,71.130524,1817000.0,A
2,2018-01-31,71.559326,2032800.0,A
3,2018-02-01,70.974609,2008200.0,A
4,2018-02-02,69.434868,1955700.0,A
...,...,...,...,...
378131,2021-01-25,161.339996,1443800.0,ZTS
378132,2021-01-26,159.710007,1982000.0,ZTS
378133,2021-01-27,153.149994,2235600.0,ZTS
378134,2021-01-28,156.529999,1859300.0,ZTS


In [302]:
merged = df_short.merge(df_marketcap, on=['symbol'], how='inner')

In [303]:
merged

,Date,Adj_Close,Volume,symbol,name,sector,marketcap
0,2018-01-29,72.631302,1321900.0,A,Agilent Technologies,Health Care,36560712369
1,2018-01-30,71.130524,1817000.0,A,Agilent Technologies,Health Care,36560712369
2,2018-01-31,71.559326,2032800.0,A,Agilent Technologies,Health Care,36560712369
3,2018-02-01,70.974609,2008200.0,A,Agilent Technologies,Health Care,36560712369
4,2018-02-02,69.434868,1955700.0,A,Agilent Technologies,Health Care,36560712369
...,...,...,...,...,...,...,...
378131,2021-01-25,161.339996,1443800.0,ZTS,Zoetis,Health Care,71792874526
378132,2021-01-26,159.710007,1982000.0,ZTS,Zoetis,Health Care,71792874526
378133,2021-01-27,153.149994,2235600.0,ZTS,Zoetis,Health Care,71792874526
378134,2021-01-28,156.529999,1859300.0,ZTS,Zoetis,Health Care,71792874526


In [305]:
merged.to_csv('preprocessed_data.csv')